# Lecture Overview

- Valuation using multiples
    - General intuition behind the approach


- Definitions of key multiples (ratios)
    - Price/Earnings (P/E)
    - Price/Book (P/B)
    - Enterprise Value / EBITDA (EV/EBITDA)


- Processing data on key ratios
    - Firm-level ratios
    - Industry-level ratios
    
    
- Valuation exercises    

In [38]:
# Import packages
import yfinance as yf
import pandas as pd
import numpy as np

# Display more rows
pd.options.display.max_rows = 500

# Pretty print all cell's output and not just the last one
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# 1. Valuation using multiples

The general idea behind using multiples to estimate the intrinsic value of a firm is that intrinsic value should have a stable relationship with some fundamental aspects of the firm (e.g. its profits or its book value). If this is the case, and we have a way to estimate this relation, then we can use it to back out the intrinsic value of the firm.

As an example, suppose we have a way to estimate the firm's true value to earnings ratio. We can then multiply this estimate by the firm's earnings, and that would give us an estimate for the firm's value. 

The most common ways of estimating the true value of some multiple for a firm are to:
- Take an average of this multiple for a handful of close competitors of the firm you are valuing
- Take an average of this multiple for the entire industry of the firm you are valuing
- Take an average over the past values of this multiple for the firm you are valuing

This is the reason why we say that multiples valuation is a form of *comparables* valuation, where comparables are firms that are similar in some fundamental way to the firm we are valuing.

We will demonstrate this technique below, using three common multiples:
- Price to earnings ratios (P/E)
- Price to book ratios (P/B, aka market to book ratios or M/B)
- (Total) Enterprise Value to EBITDA ratios (TEV/EBITDA)

# 2. Definitions for key multiples


### 2.1. The price to earnings (P/E) ratio 

The P/E ratio is calculated as the current price of the stock divided by its earnings per share (EPS). This is equivalent to dividing total market capitalization by net income. 

$$\frac{P}{E} = \frac{Price}{EPS} = \frac{Market Cap}{Net Income}$$

The P/E ratio has different names depending on which EPS (or Net Income) number we use:
- **Trailing P/E** ratios use EPS from the most recent 10-K filing
- **Current P/E** ratios use the most recent EPS available (sometimes called the TTM (trailing twelve month) EPS)
- **Forward P/E** ratios use a *forecasted* EPS


### 2.2. The price to book (P/B, or M/B) ratio

The P/B ratio is calculated as the market value of equity divided by the book value of equity:

$$\frac{P}{B} = \frac{Market Cap}{Book Equity} = \frac{Market Cap}{Assets - Liabilities}$$

High P/B firms are often called **growth firms**.

Low P/B firms are often called **value firms**.


### 2.3. (Total) Enterprise value to EBITDA (EV / EBITDA) ratio

The EV/EBITDA (or TEV/EBITDA) ratio is calculated at the enterprise value of the firm divided by the firm's EBITDA, where enterprise value is defined as the market value of the firm's equity plus the market value of the firm's debt minus cash. Often, the firm's book value of total debt is used instead of the market value of debt:

$$\frac{EV}{EBITDA} = \frac{Market Cap + Preferred Stock+ Long Term Debt - Cash}{EBITDA}$$

For long term debt, that should include the portion due within a year. 

# 3. Processing key ratios

### 3.1. Data on firm-level ratios

Please download the firm-level ratios spreadsheet for MSFT from D2L ("msft_key_ratios" under Lectures/Data) and save it in the same folder as these lecture notes (do not change the name of the file).

This spreadsheet come from https://www.morningstar.com/

If you want to download these ratios for a different company, you must create a free account first. Then you can input the ticker of your firm in "Search Quotes and Sites" at the top-left of the page. On the new page, click on the link to your firm. On the new page, hit "Key Ratios" and then "Full Key Ratios Data". On the new page, hit "Export" to download these to an excel sheet.


In [2]:
# Load data on financial ratios for MSFT
fin_ratios = pd.read_excel("./msft_key_ratios.xlsx",
                            usecols = "A:K", 
                            skiprows = [0,1],
                            nrows = 15, 
                            index_col = [0],
                            header = [0])
fin_ratios

,2011-06,2012-06,2013-06,2014-06,2015-06,2016-06,2017-06,2018-06,2019-06,2020-06
Financials,,,,,,,,,,
Revenue USD Mil,69943.00,73723.00,77849.00,86833.00,93580.00,85320.00,89950.00,110360.00,125843.00,143015.00
Gross Margin %,77.70,76.20,74.00,69.00,64.70,61.60,61.90,65.20,65.90,67.80
Operating Income USD Mil,27161.00,27956.00,26764.00,27886.00,28172.00,21292.00,22632.00,35058.00,42959.00,52959.00
Operating Margin %,38.80,37.90,34.40,32.10,30.10,25.00,25.20,31.80,34.10,37.00
Net Income USD Mil,23150.00,16978.00,21863.00,22074.00,12193.00,16798.00,21204.00,16571.00,39240.00,44281.00
Earnings Per Share USD,2.69,2.00,2.58,2.63,1.48,2.56,3.25,2.13,5.06,5.76
Dividends USD,0.61,0.76,0.89,1.07,1.21,1.39,1.53,1.65,1.80,1.99
Payout Ratio % *,22.60,26.40,44.30,38.20,49.00,101.50,66.10,109.50,39.10,32.30
Shares Mil,8593.00,8506.00,8470.00,8399.00,8254.00,8013.00,7832.00,7794.00,7753.00,7683.00


In [3]:
# Load data on income statement ratios
is_ratios = pd.read_excel("./msft_key_ratios.xlsx",
                            usecols = "A:K", 
                            skiprows = range(20),
                            nrows = 9, 
                            index_col = [0],
                            header = [0])

is_ratios 

,2011-06,2012-06,2013-06,2014-06,2015-06,2016-06,2017-06,2018-06,2019-06,2020-06
Margins % of Sales,,,,,,,,,,
Revenue,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00
COGS,22.27,23.78,26.01,31.02,35.30,38.42,38.09,34.75,34.10,32.22
Gross Margin,77.73,76.22,73.99,68.98,64.70,61.58,61.91,65.25,65.90,67.78
SG&A,25.97,24.99,26.24,23.76,21.72,22.57,22.26,20.14,18.35,17.28
R&D,12.93,13.31,13.37,13.11,12.87,14.05,14.49,13.34,13.41,13.47
Other,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Operating Margin,38.83,37.92,34.38,32.11,30.10,24.96,25.16,31.77,34.14,37.03
Net Int Inc & Other,1.30,-7.72,0.37,-0.08,-10.33,-1.81,0.57,1.28,0.58,0.05
EBT Margin,40.13,30.20,34.75,32.04,19.78,23.15,25.74,33.05,34.72,37.08


In [4]:
# Load data on profitability ratios for MSFT
profit_ratios = pd.read_excel("./msft_key_ratios.xlsx",
                            usecols = "A:K", 
                            skiprows = range(31),
                            nrows = 8, 
                            index_col = [0],
                            header = [0])

profit_ratios 

,2011-06,2012-06,2013-06,2014-06,2015-06,2016-06,2017-06,2018-06,2019-06,2020-06
Profitability,,,,,,,,,,
Tax Rate %,17.53,23.75,19.18,20.65,34.12,14.95,8.40,54.57,10.18,16.51
Net Margin %,33.10,23.03,28.08,25.42,13.03,19.69,23.57,15.02,31.18,30.96
Asset Turnover (Average),0.71,0.64,0.59,0.55,0.54,0.46,0.41,0.44,0.46,0.49
Return on Assets %,23.77,14.77,16.58,14.02,7.00,9.08,9.75,6.63,14.39,15.07
Financial Leverage (Average),1.90,1.83,1.80,1.92,2.20,2.69,3.33,3.13,2.80,2.55
Return on Equity %,44.84,27.51,30.09,26.17,14.36,22.09,29.37,21.37,42.41,40.14
Return on Invested Capital %,37.71,23.25,25.70,21.79,11.16,14.81,16.36,11.49,24.13,25.11
Interest Coverage,96.16,59.60,64.06,47.60,24.70,16.89,11.42,14.35,17.27,21.47


In [5]:
# Load balance sheet
bsheet_ratios = pd.read_excel("./msft_key_ratios.xlsx",
                            usecols = "A:K", 
                            skiprows = range(73),
                            nrows = 20, 
                            index_col = [0],
                            header = [0])


bsheet_ratios

,2011-06,2012-06,2013-06,2014-06,2015-06,2016-06,2017-06,2018-06,2019-06,2020-06
Balance Sheet Items (in %),,,,,,,,,,
Cash & Short-Term Investments,48.55,51.98,54.08,49.72,54.77,58.46,55.16,51.68,46.70,45.31
Accounts Receivable,13.79,13.01,12.28,11.34,10.16,9.44,8.21,10.23,10.30,10.62
Inventory,1.26,0.94,1.36,1.54,1.65,1.16,0.90,1.03,0.72,0.63
Other Current Assets,5.32,4.23,3.52,3.67,4.19,3.04,2.03,2.61,3.54,3.81
Total Current Assets,68.92,70.16,71.24,66.27,70.77,72.10,66.30,65.55,61.26,60.37
Net PP&E,7.51,6.82,7.01,7.55,8.36,9.48,9.84,13.96,15.30,17.56
Intangibles,12.26,13.71,12.45,15.73,12.36,11.15,18.76,16.90,17.37,16.72
Other Long-Term Assets,11.31,9.31,9.29,10.45,8.52,7.27,5.09,3.59,6.06,5.34
Total Assets,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00


In [6]:
# Load efficiency ratios
eff_ratios = pd.read_excel("./msft_key_ratios.xlsx",
                            usecols = "A:K", 
                            skiprows = range(102),
                            nrows = 8, 
                            index_col = [0],
                            header = [0])

eff_ratios

,2011-06,2012-06,2013-06,2014-06,2015-06,2016-06,2017-06,2018-06,2019-06,2020-06
Efficiency,,,,,,,,,,
Days Sales Outstanding,73.06,76.16,77.98,77.83,73.04,77.40,77.24,76.52,81.22,78.52
Days Inventory,24.74,26.12,27.71,31.16,30.72,28.69,23.61,23.05,20.10,15.68
Payables Period,96.33,87.16,81.14,83.07,77.46,75.10,76.11,76.17,76.55,86.79
Cash Conversion Cycle,1.48,15.13,24.56,25.91,26.30,30.99,24.74,23.40,24.76,7.41
Receivables Turnover,5.00,4.79,4.68,4.69,5.00,4.72,4.73,4.77,4.49,4.65
Inventory Turnover,14.75,13.97,13.17,11.72,11.88,12.72,15.46,15.84,18.16,23.28
Fixed Assets Turnover,8.86,8.97,8.53,7.55,6.75,5.16,4.27,3.68,3.14,2.96
Asset Turnover,0.71,0.64,0.59,0.55,0.54,0.46,0.41,0.44,0.46,0.49


In [7]:
# Save for later use
all_ratios = fin_ratios.append(is_ratios.append(profit_ratios.append(bsheet_ratios.append(eff_ratios))))
all_ratios.to_pickle('./msft_ratios.pkl')

#### 3.1.1 Competitor ratios

To find a firm's competitors (acording to Morningstar), go to the firm's Morningstar page (as explained above) and hit the "Quote" tab. Then hit "Competitors" (towards the middle of the page).

For MSFT, this should show AMZN, AAPL, and GOOG as the competitors. Hit "More Competitors Data" and you will see their Price/Earnings (P/E) and Price/Book (P/B) ratios.

To obtain more ratios (e.g. Enterprise Value / EBITDA), you need to go to each competitor's Morningstar page and hit the "Valuation" tab.

In [8]:
# List of competitors for MSFT
msft_comp_tickers = ['AMZN','AAPL','GOOG']

# Competitor ratios (as of 2020 i.e. "trailing" ratios)
msft_comp_pe = [95.23, 40.45, 33.38]  
msft_comp_pb = [19.79, 34.16, 5.57]
msft_comp_evebitda = [35.93, 27.82, 20.1]

# MSFT own ratios (past 10 years)
msft_own_pe = [9.44,14.68,14.01,18.22,36.74,29.73,30.23,41.80,29.75,35.87]
msft_own_pb = [3.66,3.08,3.80,4.23,5.68,6.83,7.35,9.08,11.32,13.60]
msft_own_evebitda = [5.53,7.15,7.59,9.40,15.17,15.05,16.91,13.96,18.80,22.55]

### 3.2. Data on industry-level ratios

Please download the industry-level ratios from D2L ("pedata", "pbvdata", "vebitda" files under Lectures/Data) and save them in the same folder as these lecture notes (do not change the names of these files).

These files come from Aswath Damodaran at NYU. If you want to download these data yourself, go to 

http://pages.stern.nyu.edu/~adamodar/

Then hit "Data", and from the "Current Data" menu, select "Multiples".

We will not import these Excel sheets into Python because we only need a single number from each of them so it's easier to just open the Excel files and look up the numbers we need.

In [9]:
# Industry ratios from Damoradan files (MSFT is in "Software (System & Application)")
msft_industry_pe = 39.16   # Aggregate Mkt Cap / Trailing Net Income
msft_industry_pb = 14.07
msft_industry_evebitda = 30.42

# 4. Valuation exercises

### 4.1. Using P/E ratios

#### 4.1.1.  A simple example

Assume in the most recent fiscal year, MSFT had EPS of $5.7, and its main competitors had P/E ratios of 35, 40, and 45 respectively. Estimate the intrinsic value of a share of MSFT using this data.

In [10]:
# Estimate true P/E ratio
comp_pe = [35, 40, 45]
av_pe = np.mean(comp_pe)
av_pe

40.0

In [11]:
# Solve for intrinsic value
iv_using_pe = av_pe * 5.7
iv_using_pe

228.0

#### 4.1.2. Using real-world data

We estimate the true P/E ratio of MSFT in three different ways, based on 
1. Close competitor ratios
2. Industry ratio
3. Own past ratios

We then multiply each of these estimates with the most recent EPS of MSFT to back out estimates of the true (intrinsic) value of a share of MSFT.

In [12]:
# Estimate true P/E
pe_comp = np.mean(msft_comp_pe)
pe_ind = msft_industry_pe
pe_own = np.mean(msft_own_pe[5:10]) #full msft_own_pe gives very low value

print('Estimates of MSFT true P/E ratio:\n')
print('Using close competitors:', pe_comp)
print('Using industry:', pe_ind)
print('Using own ratio from the past:', pe_own)

Estimates of MSFT true P/E ratio:

Using close competitors: 56.35333333333333
Using industry: 39.16
Using own ratio from the past: 33.476


In [13]:
# Get latest EPS
financials = pd.read_pickle('./msft_all_statements.pkl')
eps = financials.loc['Diluted EPS','2020'][0]   #see what happens without the [0]
eps

5.76

In [14]:
# Solve for instrinsic value
price_pe_comp = eps * pe_comp
price_pe_ind = eps * pe_ind
price_pe_own = eps * pe_own

print('Estimates of MSFT intrinsic value using P/E ratios:\n')
print('Using close competitors:', price_pe_comp)
print('Using industry:', price_pe_ind)
print('Using own ratio from the past:', price_pe_own)

Estimates of MSFT intrinsic value using P/E ratios:

Using close competitors: 324.5952
Using industry: 225.56159999999997
Using own ratio from the past: 192.82175999999998


Based on the above estimates, is MSFT currently a buy, sell or hold?

### 4.2. Using P/B ratios

#### 4.2.1.  A simple example

Assume in the most recent fiscal year, MSFT had book value of equity of $118 (bil) and 7.6 bil shares outstanding. Its main competitors had P/B ratios of 15, 20, and 25 respectively. Estimate the intrinsic value of a share of MSFT using this data.

In [15]:
# Estimate true P/B ratio
comp_pb = [15, 20, 25]
av_pb = np.mean(comp_pb)
av_pb

20.0

In [16]:
# Solve for intrinsic value
iv_using_pb = av_pb * 118
iv_using_pb

2360.0

#### 4.2.2. Using real-world data

We estimate the true P/B ratio of MSFT in three different ways, based on 
1. Close competitor ratios
2. Industry ratio
3. Own past ratios

We then multiply each of these estimates with the most recent book value of MSFT to back out estimates of the true (intrinsic) value of ALL shares of MSFT. We divide by the number of shares to obtain per-share estimates of the intrinsic value.

In [17]:
# Estimate true P/B
pb_comp = np.mean(msft_comp_pb)
pb_ind = msft_industry_pb
pb_own = np.mean(msft_own_pb[5:10])

print("P/B using close competitors = ", pb_comp)
print("P/B using industry = ", pb_ind)
print("P/B using own ratio from the past = ", pb_own)

P/B using close competitors =  19.84
P/B using industry =  14.07
P/B using own ratio from the past =  9.636


In [32]:
# Get latest book equity and number of shares outstanding
book_equity = financials.loc['Total Equity', '2020']
nr_shares = financials.loc['Diluted WASO', '2020']

print("Book equity in 2020:", book_equity)
print("Nr shares in 2020:", nr_shares)

Book equity in 2020: 118304.0
Nr shares in 2020: 7683.0


In [36]:
# Solve for instrinsic value
price_pb_comp = pb_comp * book_equity / nr_shares
price_pb_ind = pb_ind * book_equity / nr_shares
price_pb_own = pb_own * book_equity / nr_shares

print("Intrisic value using P/B of close competitors: ", price_pb_comp)
print("Intrisic value using P/B of the industry: ", price_pb_ind)
print("Intrisic value using P/B of MSFT from the past: ", price_pb_own)

Intrisic value using P/B of close competitors:  305.4993309904985
Intrisic value using P/B of the industry:  216.65199531433035
Intrisic value using P/B of MSFT from the past:  148.37659039437716


Based on the above estimates, is MSFT currently a buy, sell or hold?

### 4.3. Using EV/EBITDA ratios

#### 4.2.1.  A simple example

Assume in the most recent fiscal year, MSFT had EBITDA of \\$118 (bil) and 7.6 bil shares outstanding. It also had \\$98 bil in long term debt and \\$113 (bil) in cash holdings.  Its main competitors had P/B ratios of 15, 20, and 25 respectively. Estimate the intrinsic value of a share of MSFT using this data.

In [20]:
# Inputs
ebitda = 118
debt = 98
cash = 113
shares = 7.6

In [21]:
# Estimate true P/E ratio
comp_evebitda = [15, 20, 25]
av_evebitda = np.mean(comp_evebitda)
av_evebitda

20.0

In [22]:
# Solve for ENTERPRISE intrinsic value
ev_using_evebitda = av_evebitda * ebitda
ev_using_evebitda

2360.0

In [23]:
# Solve for intrinsic value (total)
mktcap_estimate = ev_using_evebitda + cash - debt
mktcap_estimate

2375.0

In [24]:
# Solve for intrinsic value per share
price_estimate = mktcap_estimate / shares
price_estimate

312.5

#### 4.2.2. Using real-world data

We estimate the true P/B ratio of MSFT in three different ways, based on 
1. Close competitor ratios
2. Industry ratio
3. Own past ratios

We then multiply each of these estimates with the most recent book value of MSFT to back out estimates of the true (intrinsic) value of ALL shares of MSFT. We divide by the number of shares to obtain per-share estimates of the intrinsic value.

In [40]:
# Estimate true EV/EBITDA
evebitda_comp = np.mean(msft_comp_evebitda)
evebitda_ind = msft_industry_evebitda
evebitda_own = np.mean(msft_own_evebitda[5:10])

print("EV/EBITDA estimate using close competitors: ", evebitda_comp)
print("EV/EBITDA estimate using industry ratio: ", evebitda_ind)
print("EV/EBITDA estimate using own ratio from the past: ", evebitda_own)

EV/EBITDA estimate using close competitors:  27.95
EV/EBITDA estimate using industry ratio:  30.42
EV/EBITDA estimate using own ratio from the past:  17.454


In [39]:
financials

,2016,2017,2018,2019,2020
Item,,,,,
Total Assets,193694.00,241086.00,258848.00,286556.00,301311.00
Total Current Assets,139660.00,159851.00,169662.00,175552.00,181915.00
"Cash, Cash Equivalents and Short Term Investments",113240.00,132981.00,133768.00,133819.00,136527.00
Cash and Cash Equivalents,6510.00,7663.00,11946.00,11356.00,13576.00
Short Term Investments,106730.00,125318.00,121822.00,122463.00,122951.00
Inventories,2251.00,2181.00,2662.00,2063.00,1895.00
"Raw Materials, Consumables and Supplies",612.00,797.00,655.00,399.00,700.00
Work-in-Process,158.00,145.00,54.00,53.00,83.00
Finished Goods and Merchandise,1481.00,1239.00,1953.00,1611.00,1112.00


In [41]:
# Get latest long term debt and cash
lt_debt_due = financials.loc['Current Portion of Long Term Debt and Capital Lease', '2020']
lt_debt = financials.loc['Long Term Debt and Capital Lease Obligation', '2020']
cash = financials.loc['Cash, Cash Equivalents and Short Term Investments', '2020']

# Get latest EBITDA
ebit = financials.loc['Total Operating Profit/Loss', '2020']
depreciation = financials.loc['Depreciation, Amortization and Depletion, Non-Cash Adjustment', '2020']
ebitda = ebit + depreciation

In [42]:
# Solve for instrinsic ENTERPRISE value
ev_comp = evebitda_comp * ebitda
ev_ind = evebitda_ind * ebitda
ev_own = evebitda_own * ebitda

print("EV using competitors: ", ev_comp)
print("EV using industry: ", ev_ind)
print("EV using own firms ratios from the past: ", ev_own)

EV using competitors:  1837852.25
EV using industry:  2000267.1
EV using own firms ratios from the past:  1147687.77


In [43]:
# Solve for instrinsic value per share
price_ev_comp = (ev_comp +cash - lt_debt - lt_debt_due) / nr_shares
price_ev_ind = (ev_ind +cash - lt_debt - lt_debt_due) / nr_shares
price_ev_own = (ev_own +cash - lt_debt - lt_debt_due) / nr_shares

print('Intrisic value using competitors: ', price_ev_comp)
print('Intrisic value using industry: ', price_ev_ind)
print('Intrisic value using own firm ratios from the past: ', price_ev_own)

Intrisic value using competitors:  247.7393270857738
Intrisic value using industry:  268.8788363920344
Intrisic value using own firm ratios from the past:  157.90925029285435


Based on the above estimates, is MSFT currently a buy, sell or hold?

In [44]:
# Save all results
results = [price_pe_own, price_pe_comp, price_pe_ind,
          price_pb_own, price_pb_comp, price_pb_ind,
          price_ev_own, price_ev_comp, price_ev_ind]
results

model_names = ['pe_own', 'pe_comp', 'pe_ind',
          'pb_own', 'pb_comp', 'pb_ind',
          'ev_own', 'ev_comp', 'ev_ind']
model_names

df = pd.DataFrame(results, index = model_names, columns = ['Intrinsic Value'])
df

df.to_pickle('./msft_valuation_multiples.pkl')

[192.82175999999998,
 324.5952,
 225.56159999999997,
 148.37659039437716,
 305.4993309904985,
 216.65199531433035,
 157.90925029285435,
 247.7393270857738,
 268.8788363920344]

['pe_own',
 'pe_comp',
 'pe_ind',
 'pb_own',
 'pb_comp',
 'pb_ind',
 'ev_own',
 'ev_comp',
 'ev_ind']

,Intrinsic Value
pe_own,192.821760
pe_comp,324.595200
pe_ind,225.561600
pb_own,148.376590
pb_comp,305.499331
pb_ind,216.651995
ev_own,157.909250
ev_comp,247.739327
ev_ind,268.878836
